# 8 - Predições da demanda


Para entender como o atendimento pode se comportar num futuro próximo, vamos usar modelos preditivos. Assim, vamos tentar estimar algumas metas da Secretaria Municipal de Educação. Vamos começar com a demanda. A meta é zerá-la. Portanto, vamos calcular quando isso acontece:

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m')
dados_educ_pop = pd.read_csv('dados/dados_educ_pop.csv', sep = ',', parse_dates=['ANO_MES'], index_col='ANO_MES', date_parser=dateparse) 

# Só precisamos do distrito e da demanda, para esse caso
dados_demanda = dados_educ_pop[['DISTRITO', 'DEM_CRECHE']]

Usando o modelo de predição ARIMA, sigla para Média móvel integrada autoregressiva, que forneceu o menor erro médio nos testes, vamos tentar descobrir em torno de qual mês a demanda atinge zero, para esse distrito. Vamos prever até no máximo 2030.

In [2]:
import statsmodels.api as sm

for distrito in dados_demanda.DISTRITO.unique():
    dados_demanda = dados_educ_pop[dados_educ_pop['DISTRITO'] == distrito]
    zero = False
    
    # Gerando o modelo
    fit1 = sm.tsa.statespace.SARIMAX(dados_demanda.DEM_CRECHE, order=(3, 2, 1), seasonal_order=(0, 1, 1, 4)).fit()
    demanda_prevista = fit1.predict(start="2018-03-01", end="2030-12-01", dynamic=True)
    
    # Verificando se a meta foi atingida
    for index, element in demanda_prevista.iteritems():
        if element <= 0:
            zero = True
            data = str(index)
            print(distrito + " - " + data[5:7] + '/' + data[:4])
            break
    if not zero:
        print("O distrito " + distrito + " só vai ter a fila zerada a partir de 2031.")
    
    # Gerando o gráfico
    plt.ioff()
    fig = plt.figure(dpi=200, figsize=(10, 6))
    plt.title('Evolução da fila das creches em ' + distrito)
    plt.plot(dados_demanda['DEM_CRECHE'], label='Demanda registrada nas creches do distrito', color='c', linewidth=5.0)
    plt.plot(demanda_prevista, label='Demanda prevista nas creches do distrito', color='midnightblue', linewidth=5.0)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),  shadow=True, ncol=2)
    plt.ylabel('Crianças de 0 a 3 anos')
    plt.xlabel('Data de Referência')
    plt.xticks(rotation=90)
    plt.savefig('graficos/previsao_demanda_' + distrito + '.png', bbox_inches='tight')
    plt.close(fig)